In [1]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import json
import requests
import time
from IPython.display import display

In [2]:
url = "dbikes.cznzccwi0urk.us-east-1.rds.amazonaws.com"
user = "admin"
database= "dbikes"
port = "3306"
password = "Foryiuxing18!"

In [3]:
engine = create_engine(f"mysql+mysqldb://{user}:{password}@{url}:{port}/{database}", echo=True)

In [4]:
for res in engine.execute("SHOW VARIABLES;"):
    print(res)

2023-04-05 18:08:12,408 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-05 18:08:12,411 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-05 18:08:12,609 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-05 18:08:12,611 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-05 18:08:12,709 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-05 18:08:12,711 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-05 18:08:12,911 INFO sqlalchemy.engine.Engine SHOW VARIABLES;
2023-04-05 18:08:12,913 INFO sqlalchemy.engine.Engine [raw sql] ()
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('admin_ssl_ca', '')
('admin_ssl_capath', '')
('admin_ssl_cert', '')
('admin_ssl_cipher', '')
('admin_ssl_crl', '')
('admin_ssl_crlpath', '')
('admin_ssl_key', '')
('admin_tls_ciphersuites', '')
('admin_tls_version', 'TLSv1.2,TLSv1.3')
('authentication_policy', '*')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment

In [5]:
sql = """create database if not exists dbikes;"""
engine.execute(sql)

2023-04-03 20:42:49,477 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-03 20:42:49,480 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-03 20:42:49,677 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-03 20:42:49,679 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-03 20:42:49,777 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-03 20:42:49,778 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-03 20:42:49,972 INFO sqlalchemy.engine.Engine create database if not exists test;
2023-04-03 20:42:49,974 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-03 20:42:50,075 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
#Creating stations table
sql ="""
CREATE TABLE IF NOT EXISTS station (
address VARCHAR(256),
banking int,
bike_stands int,
bonus int, 
contract_name varchar(256),
name varchar(256),
number int primary key,
position_lat real,
position_lng real,
status varchar(256))"""

try:
    res = engine.execute("drop table if exists station")
    res = engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-03-06 12:30:30,009 INFO sqlalchemy.engine.Engine drop table if exists station
2023-03-06 12:30:30,011 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-06 12:30:30,115 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 12:30:30,301 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS station (
address VARCHAR(256),
banking int,
bike_stands int,
bonus int, 
contract_name varchar(256),
name varchar(256),
number int primary key,
position_lat real,
position_lng real,
status varchar(256))
2023-03-06 12:30:30,303 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-06 12:30:30,437 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [7]:
#Creating availability table
sql = """
Create table if not exists availability (
number int,
available_bikes int,
available_bike_stands int,
last_update bigint,
foreign key(number) references station(number))"""

try:
    
    res = engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-03-06 12:30:37,025 INFO sqlalchemy.engine.Engine 
Create table if not exists availability (
number int,
available_bikes int,
available_bike_stands int,
last_update bigint,
foreign key(number) references station(number))
2023-03-06 12:30:37,028 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-06 12:30:37,164 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [35]:
#Creating weather table
sql = """Create table if not exists dbikes.weather (
position_lng int,
position_lat int,
description Varchar(256),
temperature int,
feels_like int, 
temp_min int, 
temp_max int, 
pressure int,
humidity int,
visibility int,
wind_speed int, 
dt bigint,
sunrise int,
sunset int)"""

try:
    
    res = engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-04-05 19:23:14,902 INFO sqlalchemy.engine.Engine drop table if exists dbikes.weather
2023-04-05 19:23:14,905 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-05 19:23:15,110 INFO sqlalchemy.engine.Engine COMMIT
2023-04-05 19:23:15,309 INFO sqlalchemy.engine.Engine Create table if not exists dbikes.weather (
position_lng int,
position_lat int,
description Varchar(256),
temperature int,
feels_like int, 
temp_min int, 
temp_max int, 
pressure int,
humidity int,
visibility int,
wind_speed int, 
dt bigint,
sunrise int,
sunset int)
2023-04-05 19:23:15,311 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-05 19:23:15,470 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
"""import requests 
import traceback
import datetime 
import time
import json

url = "https://api.jcdecaux.com/vls/v1/stations?contract=Dublin&apiKey=d56dfda32709850e6f9e533857176f085c22106e"

def write_to_file(text):
    now = datetime.datetime.now()
    with open(f"data/bikes_{now}".replace(" ", "_"), "w") as f:
        f.write(text)

def stations_to_db(text):
    stations = json.loads(text)
    print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (
            station.get('address'),
            int(station.get('banking')),
            station.get('bike_stands'),
            int(station.get('bonus')),
            station.get('contract_name'),
            station.get('name'),
            station.get('number'),
            station.get('position').get('lat'),
            station.get('position').get('lng'),
            station.get('status')
        )

        engine.execute("insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", vals)
        break
        
        
def availability_to_db(text):
    availability = json.loads(text)
    print(type(availability), len(availability))
    for available in availability:
        print(available)
        vals = (
            available.get("number"),
            available.get("available_bikes"),
            available.get("available_bike_stands"),
            available.get("last_update"))
 

        engine.execute("insert into availability values (%s, %s, %s, %s)", vals)
        
    return



def main():
    while True:
        try:
            r = requests.get(url)
            print(r, datetime.datetime.now())
            write_to_file(r.text)
            stations_to_db(r.text)
            availability_to_db(r.text)
            time.sleep(5 * 60)
        except:
            print(traceback.format_exc())

if __name__ == "__main__":
    main()
"""

<Response [200]> 2023-02-28 09:51:58.565766
<class 'list'> 114
{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 22, 'available_bikes': 7, 'status': 'OPEN', 'last_update': 1677577749000}
2023-02-28 09:52:01,234 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-02-28 09:52:01,237 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 09:52:01,763 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-02-28 09:52:01,766 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 09:52:02,279 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-02-28 09:52:02,282 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 09:52:02,810 INFO sqlalchemy.engine.Engine insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-02-28 09:52:02,813 INFO sqlalchemy.engine.Engine [raw sql] ('Smithfield Nort

2023-02-28 09:52:16,245 INFO sqlalchemy.engine.Engine [raw sql] (43, 0, 30, 1677577721000)
2023-02-28 09:52:16,350 INFO sqlalchemy.engine.Engine COMMIT
{'number': 31, 'contract_name': 'dublin', 'name': 'PARNELL STREET', 'address': 'Parnell Street', 'position': {'lat': 53.350929, 'lng': -6.265125}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 8, 'available_bikes': 12, 'status': 'OPEN', 'last_update': 1677577399000}
2023-02-28 09:52:16,962 INFO sqlalchemy.engine.Engine insert into availability values (%s, %s, %s, %s)
2023-02-28 09:52:16,965 INFO sqlalchemy.engine.Engine [raw sql] (31, 12, 8, 1677577399000)
2023-02-28 09:52:17,385 INFO sqlalchemy.engine.Engine COMMIT
{'number': 98, 'contract_name': 'dublin', 'name': 'FREDERICK STREET SOUTH', 'address': 'Frederick Street South', 'position': {'lat': 53.341515, 'lng': -6.256853}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 13, 'available_bikes': 26, 'status': 'OPEN', 'last_up

2023-02-28 09:52:31,117 INFO sqlalchemy.engine.Engine [raw sql] (45, 10, 20, 1677577668000)
2023-02-28 09:52:31,545 INFO sqlalchemy.engine.Engine COMMIT
{'number': 114, 'contract_name': 'dublin', 'name': 'WILTON TERRACE (PARK)', 'address': 'Wilton Terrace (Park)', 'position': {'lat': 53.333653, 'lng': -6.248345}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 15, 'available_bikes': 25, 'status': 'OPEN', 'last_update': 1677577802000}
2023-02-28 09:52:32,068 INFO sqlalchemy.engine.Engine insert into availability values (%s, %s, %s, %s)
2023-02-28 09:52:32,070 INFO sqlalchemy.engine.Engine [raw sql] (114, 25, 15, 1677577802000)
2023-02-28 09:52:32,168 INFO sqlalchemy.engine.Engine COMMIT
{'number': 72, 'contract_name': 'dublin', 'name': 'JOHN STREET WEST', 'address': 'John Street West', 'position': {'lat': 53.343105, 'lng': -6.277167}, 'banking': False, 'bonus': False, 'bike_stands': 31, 'available_bike_stands': 30, 'available_bikes': 1, 'status': 'OPEN', 'l

2023-02-28 09:52:40,027 INFO sqlalchemy.engine.Engine [raw sql] (77, 0, 27, 1677577380000)
2023-02-28 09:52:40,461 INFO sqlalchemy.engine.Engine COMMIT
{'number': 4, 'contract_name': 'dublin', 'name': 'GREEK STREET', 'address': 'Greek Street', 'position': {'lat': 53.346874, 'lng': -6.272976}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 0, 'available_bikes': 20, 'status': 'OPEN', 'last_update': 1677577330000}
2023-02-28 09:52:41,083 INFO sqlalchemy.engine.Engine insert into availability values (%s, %s, %s, %s)
2023-02-28 09:52:41,086 INFO sqlalchemy.engine.Engine [raw sql] (4, 20, 0, 1677577330000)
2023-02-28 09:52:41,499 INFO sqlalchemy.engine.Engine COMMIT
{'number': 49, 'contract_name': 'dublin', 'name': 'GUILD STREET', 'address': 'Guild Street', 'position': {'lat': 53.347932, 'lng': -6.240928}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 16, 'available_bikes': 24, 'status': 'OPEN', 'last_update': 1677577793000}
2023

2023-02-28 09:52:54,722 INFO sqlalchemy.engine.Engine [raw sql] (117, 39, 1, 1677577607000)
2023-02-28 09:52:55,174 INFO sqlalchemy.engine.Engine COMMIT
{'number': 8, 'contract_name': 'dublin', 'name': 'CUSTOM HOUSE QUAY', 'address': 'Custom House Quay', 'position': {'lat': 53.347884, 'lng': -6.248048}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 29, 'available_bikes': 1, 'status': 'OPEN', 'last_update': 1677577688000}
2023-02-28 09:52:55,781 INFO sqlalchemy.engine.Engine insert into availability values (%s, %s, %s, %s)
2023-02-28 09:52:55,783 INFO sqlalchemy.engine.Engine [raw sql] (8, 1, 29, 1677577688000)
2023-02-28 09:52:56,200 INFO sqlalchemy.engine.Engine COMMIT
{'number': 27, 'contract_name': 'dublin', 'name': 'MOLESWORTH STREET', 'address': 'Molesworth Street', 'position': {'lat': 53.341288, 'lng': -6.258117}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 4, 'available_bikes': 16, 'status': 'OPEN', 'last_update':

...

Traceback (most recent call last):
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 3280, in _wrap_pool_connect
    return fn()
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 310, in connect
    return _ConnectionFairy._checkout(self)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 868, in _checkout
    fairy = _ConnectionRecord.checkout(pool)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 481, in checkout
    rec._checkin_failed(err, _fairy_was_created=False)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/util/langhelpers.py", line 70, in __exit__
    compat.raise_(
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/util/compat.py", line 208, in raise_
    raise exception
  File "/

<Response [200]> 2023-02-28 11:58:31.634658
<class 'list'> 114
{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 21, 'available_bikes': 8, 'status': 'OPEN', 'last_update': 1677585474000}
Traceback (most recent call last):
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 3280, in _wrap_pool_connect
    return fn()
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 310, in connect
    return _ConnectionFairy._checkout(self)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 868, in _checkout
    fairy = _ConnectionRecord.checkout(pool)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 481, i

Traceback (most recent call last):
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 3280, in _wrap_pool_connect
    return fn()
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 310, in connect
    return _ConnectionFairy._checkout(self)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 868, in _checkout
    fairy = _ConnectionRecord.checkout(pool)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 481, in checkout
    rec._checkin_failed(err, _fairy_was_created=False)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/util/langhelpers.py", line 70, in __exit__
    compat.raise_(
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/util/compat.py", line 208, in raise_
    raise exception
  File "/

<Response [200]> 2023-02-28 12:56:55.424507
<class 'list'> 114
{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 19, 'available_bikes': 9, 'status': 'OPEN', 'last_update': 1677588885000}
Traceback (most recent call last):
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 3280, in _wrap_pool_connect
    return fn()
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 310, in connect
    return _ConnectionFairy._checkout(self)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 868, in _checkout
    fairy = _ConnectionRecord.checkout(pool)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 481, i

Traceback (most recent call last):
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 3280, in _wrap_pool_connect
    return fn()
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 310, in connect
    return _ConnectionFairy._checkout(self)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 868, in _checkout
    fairy = _ConnectionRecord.checkout(pool)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 481, in checkout
    rec._checkin_failed(err, _fairy_was_created=False)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/util/langhelpers.py", line 70, in __exit__
    compat.raise_(
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/util/compat.py", line 208, in raise_
    raise exception
  File "/

In [ ]:
#adding static station data to database
import requests 
import traceback
import datetime 
import time
import json

url = "https://api.jcdecaux.com/vls/v1/stations?contract=Dublin&apiKey=d56dfda32709850e6f9e533857176f085c22106e"
r = requests.get(url)
def write_to_file(text):
    now = datetime.datetime.now()
    with open(f"data/bikes_{now}".replace(" ", "_"), "w") as f:
        f.write(r.text)

def stations_to_db(text):
    
    stations = json.loads(text)
    print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (
            station.get('address'),
            int(station.get('banking')),
            station.get('bike_stands'),
            int(station.get('bonus')),
            station.get('contract_name'),
            station.get('name'),
            station.get('number'),
            station.get('position').get('lat'),
            station.get('position').get('lng'),
            station.get('status')
        )

        engine.execute("insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", vals)
    return
stations_to_db(r.text)


<class 'list'> 115
{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 14, 'available_bikes': 16, 'status': 'OPEN', 'last_update': 1678105776000}
2023-03-06 12:30:52,086 INFO sqlalchemy.engine.Engine insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-06 12:30:52,088 INFO sqlalchemy.engine.Engine [raw sql] ('Smithfield North', 0, 30, 0, 'dublin', 'SMITHFIELD NORTH', 42, 53.349562, -6.278198, 'OPEN')
2023-03-06 12:30:52,186 INFO sqlalchemy.engine.Engine COMMIT
{'number': 30, 'contract_name': 'dublin', 'name': 'PARNELL SQUARE NORTH', 'address': 'Parnell Square North', 'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 20, 'available_bikes': 0, 'status': 'OPEN', 'last_update': 1678105815000}
2023-03-0

2023-03-06 12:30:55,559 INFO sqlalchemy.engine.Engine [raw sql] ('Portobello Road', 0, 30, 0, 'dublin', 'PORTOBELLO ROAD', 43, 53.330091, -6.268044, 'OPEN')
2023-03-06 12:30:55,655 INFO sqlalchemy.engine.Engine COMMIT
{'number': 31, 'contract_name': 'dublin', 'name': 'PARNELL STREET', 'address': 'Parnell Street', 'position': {'lat': 53.350929, 'lng': -6.265125}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 8, 'available_bikes': 12, 'status': 'OPEN', 'last_update': 1678105534000}
2023-03-06 12:30:55,845 INFO sqlalchemy.engine.Engine insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-06 12:30:55,846 INFO sqlalchemy.engine.Engine [raw sql] ('Parnell Street', 0, 20, 0, 'dublin', 'PARNELL STREET', 31, 53.350929, -6.265125, 'OPEN')
2023-03-06 12:30:55,940 INFO sqlalchemy.engine.Engine COMMIT
{'number': 98, 'contract_name': 'dublin', 'name': 'FREDERICK STREET SOUTH', 'address': 'Frederick Street South', 'position': {'lat': 53.341515, '

2023-03-06 12:30:59,306 INFO sqlalchemy.engine.Engine [raw sql] ('Earlsfort Terrace', 0, 30, 0, 'dublin', 'EARLSFORT TERRACE', 11, 53.334295, -6.258503, 'OPEN')
2023-03-06 12:30:59,400 INFO sqlalchemy.engine.Engine COMMIT
{'number': 17, 'contract_name': 'dublin', 'name': 'GOLDEN LANE', 'address': 'Golden Lane', 'position': {'lat': 53.340803, 'lng': -6.267732}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 18, 'available_bikes': 2, 'status': 'OPEN', 'last_update': 1678105309000}
2023-03-06 12:30:59,592 INFO sqlalchemy.engine.Engine insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-06 12:30:59,595 INFO sqlalchemy.engine.Engine [raw sql] ('Golden Lane', 0, 20, 0, 'dublin', 'GOLDEN LANE', 17, 53.340803, -6.267732, 'OPEN')
2023-03-06 12:30:59,690 INFO sqlalchemy.engine.Engine COMMIT
{'number': 45, 'contract_name': 'dublin', 'name': 'DEVERELL PLACE', 'address': 'Deverell Place', 'position': {'lat': 53.351464, 'lng': -6.255265}, 'banki

2023-03-06 12:31:03,208 INFO sqlalchemy.engine.Engine [raw sql] ('Lime Street', 0, 40, 0, 'dublin', 'LIME STREET', 62, 53.346026, -6.243576, 'OPEN')
2023-03-06 12:31:03,314 INFO sqlalchemy.engine.Engine COMMIT
{'number': 5, 'contract_name': 'dublin', 'name': 'CHARLEMONT PLACE', 'address': 'Charlemont Street', 'position': {'lat': 53.330662, 'lng': -6.260177}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 22, 'available_bikes': 17, 'status': 'OPEN', 'last_update': 1678105686000}
2023-03-06 12:31:03,509 INFO sqlalchemy.engine.Engine insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-06 12:31:03,511 INFO sqlalchemy.engine.Engine [raw sql] ('Charlemont Street', 0, 40, 0, 'dublin', 'CHARLEMONT PLACE', 5, 53.330662, -6.260177, 'OPEN')
2023-03-06 12:31:03,605 INFO sqlalchemy.engine.Engine COMMIT
{'number': 97, 'contract_name': 'dublin', 'name': 'KILMAINHAM GAOL', 'address': 'Kilmainham Gaol', 'position': {'lat': 53.342113, 'lng': -6.3100

2023-03-06 12:31:07,021 INFO sqlalchemy.engine.Engine [raw sql] ('Newman House', 0, 40, 0, 'dublin', 'NEWMAN HOUSE', 53, 53.337132, -6.26059, 'OPEN')
2023-03-06 12:31:07,114 INFO sqlalchemy.engine.Engine COMMIT
{'number': 58, 'contract_name': 'dublin', 'name': "SIR PATRICK DUN'S", 'address': "Sir Patrick's Dun", 'position': {'lat': 53.339218, 'lng': -6.240642}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 8, 'available_bikes': 31, 'status': 'OPEN', 'last_update': 1678105714000}
2023-03-06 12:31:07,308 INFO sqlalchemy.engine.Engine insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-06 12:31:07,310 INFO sqlalchemy.engine.Engine [raw sql] ("Sir Patrick's Dun", 0, 40, 0, 'dublin', "SIR PATRICK DUN'S", 58, 53.339218, -6.240642, 'OPEN')
2023-03-06 12:31:07,404 INFO sqlalchemy.engine.Engine COMMIT
{'number': 66, 'contract_name': 'dublin', 'name': 'NEW CENTRAL BANK', 'address': 'New Central Bank', 'position': {'lat': 53.347122, 'lng': -

2023-03-06 12:31:10,872 INFO sqlalchemy.engine.Engine [raw sql] ('Market Street South', 0, 38, 0, 'dublin', 'MARKET STREET SOUTH', 76, 53.342296, -6.287661, 'OPEN')
2023-03-06 12:31:10,972 INFO sqlalchemy.engine.Engine COMMIT
{'number': 71, 'contract_name': 'dublin', 'name': 'KEVIN STREET', 'address': 'Kevin Street', 'position': {'lat': 53.337757, 'lng': -6.267699}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 37, 'available_bikes': 3, 'status': 'OPEN', 'last_update': 1678105493000}
2023-03-06 12:31:11,169 INFO sqlalchemy.engine.Engine insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-06 12:31:11,172 INFO sqlalchemy.engine.Engine [raw sql] ('Kevin Street', 0, 40, 0, 'dublin', 'KEVIN STREET', 71, 53.337757, -6.267699, 'OPEN')
2023-03-06 12:31:11,270 INFO sqlalchemy.engine.Engine COMMIT
{'number': 79, 'contract_name': 'dublin', 'name': 'ECCLES STREET EAST', 'address': 'Eccles Street East', 'position': {'lat': 53.358115, 'lng': -6

2023-03-06 12:31:16,006 INFO sqlalchemy.engine.Engine [raw sql] ('Townsend Street', 0, 20, 0, 'dublin', 'TOWNSEND STREET', 22, 53.345922, -6.254614, 'OPEN')
2023-03-06 12:31:16,099 INFO sqlalchemy.engine.Engine COMMIT
{'number': 50, 'contract_name': 'dublin', 'name': 'GEORGES LANE', 'address': "George's Lane", 'position': {'lat': 53.35023, 'lng': -6.279696}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 38, 'available_bikes': 2, 'status': 'OPEN', 'last_update': 1678105594000}
2023-03-06 12:31:16,291 INFO sqlalchemy.engine.Engine insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-06 12:31:16,293 INFO sqlalchemy.engine.Engine [raw sql] ("George's Lane", 0, 40, 0, 'dublin', 'GEORGES LANE', 50, 53.35023, -6.279696, 'OPEN')
2023-03-06 12:31:16,386 INFO sqlalchemy.engine.Engine COMMIT
{'number': 110, 'contract_name': 'dublin', 'name': 'PHIBSBOROUGH ROAD', 'address': 'Phibsborough Road', 'position': {'lat': 53.356307, 'lng': -6.273717},

2023-03-06 12:32:37,733 INFO sqlalchemy.engine.Engine [raw sql] ('Parkgate Street', 0, 38, 0, 'dublin', 'PARKGATE STREET', 86, 53.347972, -6.291804, 'OPEN')
2023-03-06 12:32:37,826 INFO sqlalchemy.engine.Engine COMMIT
{'number': 35, 'contract_name': 'dublin', 'name': 'SMITHFIELD', 'address': 'Smithfield', 'position': {'lat': 53.347692, 'lng': -6.278214}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 14, 'available_bikes': 15, 'status': 'OPEN', 'last_update': 1678105334000}
2023-03-06 12:32:38,018 INFO sqlalchemy.engine.Engine insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-06 12:32:38,020 INFO sqlalchemy.engine.Engine [raw sql] ('Smithfield', 0, 30, 0, 'dublin', 'SMITHFIELD', 35, 53.347692, -6.278214, 'OPEN')
2023-03-06 12:32:38,114 INFO sqlalchemy.engine.Engine COMMIT
{'number': 10, 'contract_name': 'dublin', 'name': 'DAME STREET', 'address': 'Dame Street', 'position': {'lat': 53.344007, 'lng': -6.266802}, 'banking': True, 'b

In [ ]:
sql = "select * from dbikes.station limit 10"
for row in engine.execute(sql):
    print(row)

In [41]:
#adding avaiability data to database
url = "https://api.jcdecaux.com/vls/v1/stations?contract=Dublin&apiKey=d56dfda32709850e6f9e533857176f085c22106e"
r = requests.get(url)
def write_to_file(text):
    now = datetime.datetime.now()
    with open(f"data/bikes_{now}".replace(" ", "_"), "w") as f:
        f.write(r.text)
        
def availability_to_db(text):
    availability = json.loads(text)
    print(type(availability), len(availability))
    for available in availability:
        print(available)
        vals = (
            available.get("number"),
            available.get("available_bikes"),
            available.get("available_bike_stands"),
            available.get("last_update"))
 

        engine.execute("insert into availability values (%s, %s, %s, %s)", vals)
        
    return
availability_to_db(r.text)

2023-03-04 12:06:45,444 INFO sqlalchemy.engine.Engine update availability set available_bikes = %s, available_bike_stands = %s, last_update = %s where number = %s
2023-03-04 12:06:45,446 INFO sqlalchemy.engine.Engine [raw sql] (29, 1, 1677931475000, 42)
2023-03-04 12:06:45,699 INFO sqlalchemy.engine.Engine COMMIT
2023-03-04 12:06:45,957 INFO sqlalchemy.engine.Engine update availability set available_bikes = %s, available_bike_stands = %s, last_update = %s where number = %s
2023-03-04 12:06:45,960 INFO sqlalchemy.engine.Engine [raw sql] (7, 13, 1677931133000, 30)
2023-03-04 12:06:46,124 INFO sqlalchemy.engine.Engine COMMIT
2023-03-04 12:06:46,482 INFO sqlalchemy.engine.Engine update availability set available_bikes = %s, available_bike_stands = %s, last_update = %s where number = %s
2023-03-04 12:06:46,484 INFO sqlalchemy.engine.Engine [raw sql] (7, 26, 1677931382000, 54)
2023-03-04 12:06:46,743 INFO sqlalchemy.engine.Engine COMMIT
2023-03-04 12:06:46,989 INFO sqlalchemy.engine.Engine u

2023-03-04 12:06:58,281 INFO sqlalchemy.engine.Engine update availability set available_bikes = %s, available_bike_stands = %s, last_update = %s where number = %s
2023-03-04 12:06:58,284 INFO sqlalchemy.engine.Engine [raw sql] (2, 18, 1677931291000, 17)
2023-03-04 12:06:58,436 INFO sqlalchemy.engine.Engine COMMIT
2023-03-04 12:06:59,001 INFO sqlalchemy.engine.Engine update availability set available_bikes = %s, available_bike_stands = %s, last_update = %s where number = %s
2023-03-04 12:06:59,003 INFO sqlalchemy.engine.Engine [raw sql] (9, 21, 1677931103000, 45)
2023-03-04 12:06:59,161 INFO sqlalchemy.engine.Engine COMMIT
2023-03-04 12:06:59,561 INFO sqlalchemy.engine.Engine update availability set available_bikes = %s, available_bike_stands = %s, last_update = %s where number = %s
2023-03-04 12:06:59,563 INFO sqlalchemy.engine.Engine [raw sql] (9, 31, 1677931276000, 114)
2023-03-04 12:06:59,724 INFO sqlalchemy.engine.Engine COMMIT
2023-03-04 12:06:59,968 INFO sqlalchemy.engine.Engine 

2023-03-04 12:07:11,388 INFO sqlalchemy.engine.Engine update availability set available_bikes = %s, available_bike_stands = %s, last_update = %s where number = %s
2023-03-04 12:07:11,391 INFO sqlalchemy.engine.Engine [raw sql] (15, 25, 1677931449000, 58)
2023-03-04 12:07:11,542 INFO sqlalchemy.engine.Engine COMMIT
2023-03-04 12:07:11,913 INFO sqlalchemy.engine.Engine update availability set available_bikes = %s, available_bike_stands = %s, last_update = %s where number = %s
2023-03-04 12:07:11,915 INFO sqlalchemy.engine.Engine [raw sql] (0, 40, 1677931182000, 66)
2023-03-04 12:07:12,054 INFO sqlalchemy.engine.Engine COMMIT
2023-03-04 12:07:12,304 INFO sqlalchemy.engine.Engine update availability set available_bikes = %s, available_bike_stands = %s, last_update = %s where number = %s
2023-03-04 12:07:12,307 INFO sqlalchemy.engine.Engine [raw sql] (5, 35, 1677931230000, 104)
2023-03-04 12:07:12,464 INFO sqlalchemy.engine.Engine COMMIT
2023-03-04 12:07:12,708 INFO sqlalchemy.engine.Engine

2023-03-04 12:07:23,971 INFO sqlalchemy.engine.Engine update availability set available_bikes = %s, available_bike_stands = %s, last_update = %s where number = %s
2023-03-04 12:07:23,973 INFO sqlalchemy.engine.Engine [raw sql] (9, 31, 1677931300000, 50)
2023-03-04 12:07:24,120 INFO sqlalchemy.engine.Engine COMMIT


Exception during reset or similar
Traceback (most recent call last):
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "/Users/oghenemaluighoiye/opt/anaconda3/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 682, in do_rollback
    dbapi_connection.rollback()
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
#update availability
url = "https://api.jcdecaux.com/vls/v1/stations?contract=Dublin&apiKey=d56dfda32709850e6f9e533857176f085c22106e"
r = requests.get(url)



def write_to_file(text):
    now = datetime.datetime.now()
    with open(f"data/bikes_{now}".replace(" ", "_"), "w") as f:
        f.write(r.text)

def update_availability(text):
    availability = json.loads(text)
    for available in availability:
        
        engine.execute("update availability set available_bikes = %s, available_bike_stands = %s, last_update = %s where number = %s", (available.get("available_bikes"), available.get("available_bike_stands"), available.get("last_update"), available.get("number")))

    return


def main():
    while True:
        try:
            r = requests.get(url)
            print(r, datetime.datetime.now())
            write_to_file(r.text)
            update_availability(r.text)
            time.sleep(5 * 60)
        except:
            print(traceback.format_exc())

if __name__ == "__main__":
    main()


In [1]:
#scraping weather data to database
url = "https://api.openweathermap.org/data/2.5/weather?q=Dublin,IE&units=metric&appid=925fb7d8a523499058239184ca7054d9"
import datetime
def write_to_file2(text):
    now = datetime.datetime.now()
    with open(f"data/weather_{now}".replace(" ", "_"), "w") as f:
        f.write(text) 
        
def weather_to_db(text):
    weather = json.loads(text)
    print(type(weather),len(weather))
    print(text)
    now = datetime.datetime.now()
    vals = (
            weather.get("coord").get("lon"),
            weather.get("coord").get("lat"),
            weather.get("weather")[0].get("description"),
            weather.get("main").get("temp"),
            weather.get("main").get("feels_like"),
            weather.get("main").get("temp_min"),
            weather.get("main").get("temp_max"),
            weather.get("visibility"),
            weather.get("wind").get("speed"),
            weather.get("sys").get("sunrise"),
            weather.get("sys").get("sunrise"))
        
    
    
    print("\n\n\nVALS:{}\n\n\n".format(vals))
    engine.execute("insert into weather values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", vals)
    
def weather_main():
    
    try:
        url = "https://api.openweathermap.org/data/2.5/weather?q=Dublin,IE&units=metric&appid=925fb7d8a523499058239184ca7054d9"

        weather = requests.get(url)
        write_to_file2(weather.text)
        weather_to_db(weather.text)
    except:
          print(traceback.format_exc())
          if engine is None:
                return

if __name__ == "__main__":
    weather_main() 


NameError: name 'traceback' is not defined